<a id='20'></a>
<font color = '#F08841'>
    Content:

1. [Import library and data](#0)    
1. [Data Processing](#1)
1. [Data Visualization](#2)
    *          [Check for Outliers](#3)
    *          [Checking Relationship Amoung Features](#4)
    *          [Plotting Columns against Date_Time](#5)
1.  [Conclusion After EDA](#6)
1.  [Feature Engineering](#7)
2.  [Model Creation](#8)
3.  [Prediction](#9)
4.  [Submittion](#10)   
5.  [Glossery](#15)

<a id="0"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Importing Required Libraries and Data </strong></p>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from scipy.stats import ttest_rel

import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')


<a id="1"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong> Data Processing</strong></p>


In [ ]:
df

* First thing which we observe is it has hourly data if air quality. which we might need to parse

In [ ]:
# Parsing Date
# create a new column, date_parsed, with the parsed dates

df['date_time'] = pd.to_datetime(df['date_time'])

'''df['Date'] = df['date_parsed'].dt.strftime('%m/%d/%y/')
df['Time'] = df['date_parsed'].dt.strftime('%H:%M')'''

df.head(2)

In [ ]:
df.index=df['date_time']
df.drop(columns= ['date_time'], axis = 1, inplace=True)


In [ ]:
df.describe()

In [ ]:
df.info()

<a id="2"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Data Visualization</strong></p>


<a id="3"></a>
<p style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ffb84d; border-radius: 5px 5px;"><i>Checking for outliers</i></p>


In [ ]:
df.hist(figsize=(20,20), bins=25);

In [ ]:
# Checking for outlier
# df.boxplot()

plt.figure(figsize=(20,10))
sns.boxplot(data=df)

In [ ]:
Zscore=pd.DataFrame()

Zscore['sensor1']=( df.sensor_1 -df.sensor_1.mean())/df.sensor_1.std()
Zscore['sensor2']=( df.sensor_2 -df.sensor_2.mean())/df.sensor_2.std()
Zscore['sensor3']=( df.sensor_3 -df.sensor_3.mean())/df.sensor_3.std()
Zscore['sensor4']=( df.sensor_4 -df.sensor_4.mean())/df.sensor_4.std()
Zscore['sensor5']=( df.sensor_5 -df.sensor_5.mean())/df.sensor_5.std()

Zscore['temp']=( df.deg_C -df.deg_C.mean())/df.deg_C.std()
Zscore['absolute_humidity']=( df.absolute_humidity -df.absolute_humidity.mean())/df.absolute_humidity.std()
Zscore['relative_humidity']=( df.relative_humidity -df.relative_humidity.mean())/df.relative_humidity.std()

Zscore['target_carbon_monoxide']=( df.target_carbon_monoxide -df.target_carbon_monoxide.mean())/df.target_carbon_monoxide.std()
Zscore['target_benzene']=( df.target_benzene -df.target_benzene.mean())/df.target_benzene.std()
Zscore['target_nitrogen_oxides']=( df.target_nitrogen_oxides -df.target_nitrogen_oxides.mean())/df.target_nitrogen_oxides.std()



In [ ]:
plt.figure(figsize=(15,6))


plt.subplot(1,2,1)
plt.bar(Zscore.columns, (Zscore>3).sum())
plt.xticks(rotation=90,fontsize = 15);
plt.title('ZScore > 3' , fontsize = 18,fontweight = 'bold');

plt.subplot(1,2,2)
plt.bar(Zscore.columns, (Zscore<-2.5).sum())
plt.xticks(rotation=90,fontsize = 15);
plt.title('ZScore < -2.5', fontsize = 18,fontweight = 'bold');

In [ ]:
print('No of Rows having ZScore greater than 3 is' ,(Zscore>3).sum().sum())

print('No of Rows having ZScore less than -2.5 is' ,(Zscore<-2.5).sum().sum())

If we remove all rows with Z Score less than -2.5 and greater than 3 we will loss somewhere around 12% of our Data.

<a id="4"></a>
<p style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ffb84d; border-radius: 5px 5px;"><i>Checking for Relation Amoung Features</i></p>


In [ ]:
# Check for correlation

plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot= True)


In [ ]:
# Now Checking the coveriance
# to check how the model is related
df.cov()

In [ ]:
# Checking the p value
pvalue= ttest_rel(df['relative_humidity'] , df['absolute_humidity'])
print(pvalue)

<p style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ffb84d; border-radius: 5px 5px;"><strong>Mutual Information</strong></p>

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
df_benz=pd.DataFrame()
df_no2=pd.DataFrame()
df_co = pd.DataFrame()

df_benz=df.drop(columns={'target_carbon_monoxide', 'target_nitrogen_oxides'}, axis=1)
df_co=df.drop(columns={ 'target_benzene', 'target_nitrogen_oxides'}, axis=1)
df_no2=df.drop(columns={'target_carbon_monoxide', 'target_benzene'}, axis=1)


In [ ]:
# Mutual Information for benzene


X = df_benz.copy()
y = X.pop("target_benzene")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_benz =mi_scores # Created a mi score card for benz

# Mutual Information for nitrogen Oxides


X = df_no2.copy()
y = X.pop("target_nitrogen_oxides")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_no2 =mi_scores # Created a mi score card for no2


# Mutual Information for Carbon Monoxide

X = df_co.copy()
y = X.pop("target_carbon_monoxide")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_co= mi_scores # Created a mi score card for co



In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.grid(True)


plt.figure(dpi=100, figsize=(16, 5))
plt.suptitle('Mutual Information Score', fontsize=24)

plt.subplot(1,3,1)
plot_mi_scores(ms_co)
plt.title('Carbon Monoxide')

plt.subplot(1,3,2)
plot_mi_scores(ms_no2)
plt.title('Nitrogen oxide')

plt.subplot(1,3,3)
plot_mi_scores(ms_benz)
plt.title('Benzene')

plt.tight_layout()

<a id="5"></a>
<p style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ffb84d; border-radius: 5px 5px;"><strong>Plotting Features vs Date_Time</strong></p>


In [ ]:
plt.figure(figsize=(20,20))

plt.subplot(5, 1, 1)
df['sensor_1'].plot( color='orange' , title='Sensor1')

plt.subplot(5, 1, 2)
df['sensor_2'].plot(color='red' , title="Sensor2")

plt.subplot(5, 1, 3)
df['sensor_3'].plot(color='purple' , title="Sensor3")

plt.subplot(5, 1, 4)
df['sensor_4'].plot(color='green' , title="Sensor4")

plt.subplot(5, 1, 5)
df['sensor_5'].plot(color='blue' , title="Sensor5")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(20,13))

plt.subplot(3, 1, 1)
df['target_carbon_monoxide'].plot( color='orange' , title='target_carbon_monoxide')

plt.subplot(3, 1, 2)
df['target_benzene'].plot(color='red' , title="target_benzene")

plt.subplot(3, 1, 3)
df['target_nitrogen_oxides'].plot(color='purple' , title="target_nitrogen_oxides")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(20,10))

plt.subplot(3,1,1)
df['relative_humidity'].plot(title= 'Relative_humidity')

plt.subplot(3,1,2)
df['absolute_humidity'].plot(title= 'absolute_humidity')

plt.subplot(3,1,3)
df['deg_C'].plot(title= 'Temperature in C')
plt.tight_layout()

<a id="6"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Conclusion of EDA </strong></p>
	
<p><ul style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : left; background-color : #ffe0b3; border-radius: 5px 5px;"><li>There is around 10% outliers in the data frame</li><li>Sensor1,2 & 5 are +vely correlated with all 3 Target Features</li><li>While Sensor 3 -vely correlated with all 3 Target Features</li><li>Mutual Information of <b>Target Benzene</b> sensor_2 :1.998199 sensor_3 : 0.909053
sensor_5 :            0.834663
sensor_1 :            0.778104
sensor_4 :            0.673297</li>
    <li>Mutual Information of <b>Target CO</b> sensor_2             1.040071
sensor_1             0.745881
sensor_5             0.683925
sensor_3             0.616273
sensor_4             0.386446</li>
    <li> Mutual Information of <b>Target NO2</b> sensor_2             0.551770
Date                 0.533851
sensor_5             0.529259
sensor_3             0.486198
sensor_1             0.374835
Time                 0.193417
sensor_4             0.190438
    </ul></p>


<a id="7"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Feature Engineering</strong></p>

<p style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : left; background-color : #ffe0b3; border-radius: 5px 5px;"><b>Through Mutual Information we have noticed that data and time have some effect on Target value. Thus lets create few features using date and time</b> <ul style = "font-size : 15px; color : black ; font-family : 'Comic Sans MS'; text-align : left; background-color : #ffe0b3; border-radius: 5px 5px;">
    <li><b>Season</b> (As season have effect on air pollution</li>
    <li><b>Parts of day {Phase}</b> ( pollution may increase or decrease depending on part of day, generally in morning and evening pollution due to heavy transportation rises)</li>
    <li><b>Ratio</b> <li>between relative humidity and temperature</li> <li> between absolute humidity and relative humidity </li>
    

In [ ]:
#  Preserving the original df
df_original=df

In [ ]:
# extracting features using datetime
def datetime2features(df):
    time_col = "date_time"
    df["year"] = df[time_col].dt.year
    df["month"] = df[time_col].dt.month
    df["day"] = df[time_col].dt.day
    df["hour"] = df[time_col].dt.hour
    df["dayofweek"] = df[time_col].dt.dayofweek
    df["year"] = df[time_col].dt.year
    df['weekend'] = df[time_col].dt.dayofweek.apply(lambda x: 1 if (x>4)  else 0)

In [ ]:
'''
    which phase of day the time denotes [morning, afternoon, evening, night] 
'''
def time_phase(df):
    def which_phase(hour):
        if hour >= 0 and hour <= 5:
            return 1
        elif hour >=6 and hour <= 11:
            return 2
        elif hour >=12  and hour <= 17:
            return 3
        elif hour >=18 and hour <= 23:
            return 4
        return NaN 
    time_col = "date_time"
    df["phase"] = df[time_col].dt.hour.apply(lambda x : which_phase(x))

In [ ]:
'''
    which season of year the time denotes [summer, rainy, winter] 
'''
def season(df):
    def which_season(month):
        if month >= 3 and month <= 6:
            return 1
        elif month >= 7 and month <= 9:
            return 2
        elif month >= 10  and month <= 12:
            return 3
        elif month < 3:
            return 3
        return NaN
    time_col = "date_time"
    df["season"] = df[time_col].dt.month.apply(lambda x : which_season(x))

In [ ]:
'''
ratio between absolutr_humidity and relative humidity
'''
def SMC(df):
    df['SMC'] = (df['absolute_humidity'] * 100) / df['relative_humidity']




In [ ]:
'''
ratio between relative humidity and temperature
'''
def ratio_rh_temp(df):
    df["r_rh_temp"] = df["relative_humidity"]/(df["deg_C"]+1e-9)

In [ ]:
'''
Dew Point
'''
def Dew_Point(df):
    df['Dew_Point'] = 243.12*(np.log(df['relative_humidity'] * 0.01) + (17.62 * df['deg_C'])/(243.12+df['deg_C']))/(17.62-(np.log(df['relative_humidity'] * 0.01)+17.62*df['deg_C']/(243.12+df['deg_C'])))

In [ ]:
# Doing feature engineering to df's copy
dff=df.reset_index()

datetime2features(dff)
time_phase(dff)
season(dff)
SMC(dff)
ratio_rh_temp(dff)
Dew_Point(dff)
dff.head(3)

In [ ]:
def date_time_encoding(f_name, f_itself, max_val, key):
    if (key == 'test'):
        test_data['sin_' + f_name] = np.sin(2 * np.pi * (f_itself/max_val))
        test_data['cos_' + f_name] = np.cos(2 * np.pi * (f_itself/max_val))
        test_data['tan_' + f_name] = np.tan(2 * np.pi * (f_itself/max_val))
        #test_data['sinh_' + f_name]= np.sinh(2 * np.pi * (f_itself/max_val))
        #test_data['cosh_' + f_name]= np.cosh(2 * np.pi * (f_itself/max_val))
    if (key == 'train'):
        dff['sin_' + f_name] = np.sin(2 * np.pi * (f_itself/max_val))
        dff['cos_' + f_name] = np.cos(2 * np.pi * (f_itself/max_val))
        dff['tan_' + f_name] = np.tan(2 * np.pi * (f_itself/max_val))
        #X_train['sinh_' + f_name]= np.sinh(2 * np.pi * (f_itself/max_val))
        #X_train['cosh_' + f_name]= np.cosh(2 * np.pi * (f_itself/max_val))
    return 0


In [ ]:
date_time = pd.to_datetime(dff['date_time'])
month = pd.DataFrame(date_time.dt.month)
day  = pd.DataFrame(date_time.dt.day)
hour = pd.DataFrame(date_time.dt.hour)


In [ ]:
date_time_encoding('time', hour, 24, 'train')
date_time_encoding('month', month, 12, 'train')
date_time_encoding('day' , day,  31, 'train')

## Checking correlation and mi for new df

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(dff.corr(), annot=True);

In [ ]:
df1=dff.set_index('date_time')
df1=dff.drop(columns={'date_time'}, axis=1)
df_benz=df1.drop(columns={'target_carbon_monoxide', 'target_nitrogen_oxides'}, axis=1)
df_co=df1.drop(columns={ 'target_benzene', 'target_nitrogen_oxides'}, axis=1)
df_no2=df1.drop(columns={'target_carbon_monoxide', 'target_benzene'}, axis=1)


In [ ]:
# Mutual Information for benzene


X = df_benz.copy()
y = X.pop("target_benzene")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_benz =mi_scores # Created a mi score card for benz

# Mutual Information for nitrogen Oxides


X = df_no2.copy()
y = X.pop("target_nitrogen_oxides")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_no2 =mi_scores # Created a mi score card for no2


# Mutual Information for Carbon Monoxide

X = df_co.copy()
y = X.pop("target_carbon_monoxide")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
ms_co= mi_scores # Created a mi score card for co



In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores, color='blue')
    plt.yticks(width, ticks)
    plt.grid(True)


plt.figure(dpi=100, figsize=(16, 5))
plt.suptitle('Mutual Information Score after Feature Engineering', fontsize=24)

plt.subplot(1,3,1)
plot_mi_scores(ms_co)
plt.title('Carbon Monoxide')

plt.subplot(1,3,2)
plot_mi_scores(ms_no2)
plt.title('Nitrogen oxide')

plt.subplot(1,3,3)
plot_mi_scores(ms_benz)
plt.title('Benzene')

plt.tight_layout()

<a id="8"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Model Creation</strong></p>

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, LogisticRegression
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cbt
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import pickle

In [ ]:
# Cross validation utility
class CrossValidation:
    def __init__(self, df, shuffle,random_state=None):
        self.df = df
        self.random_state = random_state
        self.shuffle = shuffle
        if shuffle is True:
            self.df = df.sample(frac=1,
                random_state=self.random_state).reset_index(drop=True)
        if not shuffle:
            self.random_state = None

    def hold_out_split(self,percent,stratify=None):
        if stratify is not None:
            y = self.df[stratify]
            train,val = ms.train_test_split(self.df, test_size=percent/100,
                stratify=y, random_state=self.random_state)
            return train,val
        size = len(self.df) - int(len(self.df)*(percent/100))
        train = self.df.iloc[:size,:]
        val = self.df.iloc[size:,:]
        return train,val

    def kfold_split(self, splits, stratify=None):
        if stratify is not None:
            kf = ms.StratifiedKFold(n_splits=splits, 
                random_state=self.random_state)
            y = self.df[stratify]
            for train, val in kf.split(X=self.df,y=y):
                t = self.df.iloc[train,:]
                v = self.df.iloc[val, :]
                yield t,v
        else:
            kf = ms.KFold(n_splits=splits, shuffle=self.shuffle,
                random_state=self.random_state)
            for train, val in kf.split(X=self.df):
                t = self.df.iloc[train,:]
                v = self.df.iloc[val, :]
                yield t,v

In [ ]:
# calculate rmsle of predicted data 
def mse(y_true, y_pred):
    return metrics.mean_squared_error(y_true, y_pred)

In [ ]:
folds = 5
seed = 48

In [ ]:
features_exclude = ["date_time"]
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']


In [ ]:
features = [col for col in dff.columns if col not in features_exclude+targets]
print(features)

### Splitting data into folds for cross validation

In [ ]:
cv = CrossValidation(dff, shuffle=True, random_state=seed)

In [ ]:
fold_models = {tar:[] for tar in targets}
print(fold_models)

# Training

In [ ]:
m_rfg = RandomForestRegressor(n_estimators=100)
m_gb = GradientBoostingRegressor(n_estimators=100)
m_lgb = lgb.LGBMRegressor(seed=seed)
m_ctb = cbt.CatBoostRegressor(random_seed=seed, verbose=False)
m_xgb = xgb.XGBRegressor(random_state=seed)

learners = (m_rfg, m_gb ,m_lgb, m_ctb, m_xgb)

meta_model = BayesianRidge(normalize=True)

In [ ]:
def train_step(X, Y, evalX, evalY, learners, meta_model, verbose=True):
    reg = StackingCVRegressor(regressors=learners, 
                            meta_regressor = meta_model,
                            n_jobs = -1,
                            verbose = int(verbose)
                           )
    trainX = X.values
    trainY = Y.values
    
    model = reg.fit(trainX, trainY)
    
    predY_train = model.predict(trainX)
    
    train_rmsle = mse(trainY, predY_train)
    train_r2 = metrics.r2_score(trainY, predY_train)
    
    if verbose:
        print("Training mse: ", train_rmsle)
        print("Training r2: ", train_r2)

    valX = evalX.values
    valY = evalY.values
    
    predY_val = model.predict(valX)
    
    val_rmsle = mse(valY, predY_val)
    val_r2 = metrics.r2_score(valY, predY_val)
    
    if verbose:
        print("Validation mse: ", val_rmsle)
        print("Validation r2: ", val_r2)
        
    return {"model": model,
            "train_scores":{"r2": train_r2, "mse": train_rmsle},
            "val_scores":{"r2": val_r2, "mse": val_rmsle}
           }


In [ ]:
def train_folds(cv, feature_cols, target_col, num_folds, learners, meta_model,
                verbose=True):
    fold_train_rmsle = []
    fold_train_r2 = []
    fold_val_rmsle = [] 
    fold_val_r2 = []
    fold_models = []

    for fold, (train_, val_) in enumerate(cv.kfold_split(splits=num_folds)):
        result = train_step(X=train_[feature_cols],
                            Y=train_[target_col],
                            evalX=val_[feature_cols],
                            evalY=val_[target_col],
                            learners=learners,
                            meta_model=meta_model,
                            verbose=verbose
                           )
        fold_train_rmsle.append(result["train_scores"]["mse"])
        
        fold_train_r2.append(result["train_scores"]["r2"])

        fold_val_rmsle.append(result["val_scores"]["mse"])
        fold_val_r2.append(result["val_scores"]["r2"])

        fold_models.append(result["model"])
        
    return {"models":fold_models,
            "train_scores":{"r2": np.mean(fold_train_r2), "mse": np.mean(fold_train_rmsle)},
            "val_scores":{"r2":np.mean(fold_val_r2), "mse":np.mean(fold_val_rmsle)}
           }


### Carbob Monoxide

In [ ]:
target = "target_carbon_monoxide"
results = train_folds(cv, features, target, folds, learners, meta_model)
fold_models[target] = results["models"]

print("="*50)
print("Training MSE: ", results["train_scores"]["mse"])
print("Training R2: ", results["train_scores"]["r2"])
print("Validation MSE: ", results["val_scores"]["mse"])
print("Validation R2: ", results["val_scores"]["r2"])

### Benzene

In [ ]:
target = "target_benzene"
results = train_folds(cv, features, target, folds, learners, meta_model)
fold_models[target] = results["models"]
print("="*50)
print("Training MSE: ", results["train_scores"]["mse"])
print("Training R2: ", results["train_scores"]["r2"])
print("Validation MSE: ", results["val_scores"]["mse"])
print("Validation R2: ", results["val_scores"]["r2"])

### Nitrogen Oxide

In [ ]:
target = "target_nitrogen_oxides"
results = train_folds(cv, features, target, folds, learners, meta_model)
fold_models[target] = results["models"]
print("="*50)
print("Training MSE: ", results["train_scores"]["mse"])
print("Training R2: ", results["train_scores"]["r2"])
print("Validation MSE: ", results["val_scores"]["mse"])
print("Validation R2: ", results["val_scores"]["r2"])

### Prediction blending from folds

In [ ]:
def get_weights(predictions, targets, apply_softmax=True):
    def softmax(x):
        f_x = np.exp(x) / np.sum(np.exp(x))
        return f_x
    lnr = LinearRegression()
    lnr_model = lnr.fit(predictions, targets)
    if apply_softmax:
        return softmax(lnr_model.coef_)
    return lnr_model.coef_

In [ ]:
def weighted_sum(predictions, weights):
    return np.dot(predictions, weights)


In [ ]:
trainX = dff[features].values
trainY = dff[targets]

In [ ]:
predictions = []

In [ ]:
preds = []
for model in fold_models[targets[0]]:
    preds.append(model.predict(trainX))

preds = np.array(preds)

weights_0 = get_weights(preds.transpose(), trainY[targets[0]].values)
print("Fold Predictions Weightings")
print(weights_0)

preds = weighted_sum(preds.transpose(), weights_0)
predictions.append(preds)

In [ ]:
preds = []
for model in fold_models[targets[1]]:
    preds.append(model.predict(trainX))

preds = np.array(preds)
weights_1 = get_weights(preds.transpose(), trainY[targets[1]].values)
print("Fold Predictions Weightings")
print(weights_1)

preds = weighted_sum(preds.transpose(), weights_1)
predictions.append(preds)

In [ ]:
preds = []
for model in fold_models[targets[2]]:
    preds.append(model.predict(trainX))

preds = np.array(preds)
weights_2 = get_weights(preds.transpose(), trainY[targets[2]].values)
print("Fold Predictions Weightings")
print(weights_2)

preds = weighted_sum(preds.transpose(), weights_2)
predictions.append(preds)

In [ ]:
predictions = np.array(predictions).transpose()
print(predictions.shape)
print(trainY.shape)

In [ ]:
predictions = np.where(predictions<0, 0, predictions)

In [ ]:
print("R2 score: ", metrics.r2_score(trainY, predictions))
print("RMSLE score: ", np.sqrt(metrics.mean_squared_log_error(trainY, predictions)))

## Inference on test dataset

In [ ]:
# import test data
df_test= pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
df_test['date_time'] = pd.to_datetime(df_test["date_time"])
df_test.head(2)

In [ ]:
# Feature engineering on test data

datetime2features(df_test)
time_phase(df_test)
season(df_test)
SMC(df_test)
ratio_rh_temp(df_test)
Dew_Point(df_test)
dff.head(2)

In [ ]:
testX = df_test[features].values

In [ ]:
predictions = []

In [ ]:
preds = []
for model in fold_models[targets[0]]:
    preds.append(model.predict(testX))

preds = np.array(preds)

preds = weighted_sum(preds.transpose(), weights_0)
predictions.append(preds)

In [ ]:
preds = []
for model in fold_models[targets[1]]:
    preds.append(model.predict(testX))

preds = np.array(preds)

preds = weighted_sum(preds.transpose(), weights_1)
predictions.append(preds)

In [ ]:
preds = []
for model in fold_models[targets[2]]:
    preds.append(model.predict(testX))

preds = np.array(preds)

preds = weighted_sum(preds.transpose(), weights_2)
predictions.append(preds)

In [ ]:
predictions = np.array(predictions).transpose()

In [ ]:
predictions.shape

In [ ]:
predictions = np.where(predictions<0, 0, predictions)

<a id="15"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Submission</strong></p>

In [ ]:
df_submission=pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')
df_submission.head(2)

In [ ]:

df_submission[targets] = predictions
df_submission.to_csv("submission.csv", index=False)
df_submission.head()

<a id="15"></a>
<p style = "font-size : 20px; color : black ; font-family : 'Comic Sans MS'; text-align : center; background-color : #ff6666; border-radius: 5px 5px;"><strong>Glossery</strong></p>
# Glossery

* Absolute humidity is the measure of water vapor (moisture) in the air, regardless of temperature. It is expressed as grams of moisture per cubic meter of air (g/m3).

* Relative humidity also measures water vapor but RELATIVE to the temperature of the air. It is expressed as the amount of water vapor in the air as a percentage of the total amount that could be held at its current temperature.Warm air can hold far more moisture than cold air meaning that the relative humidity of cold air would be far higher than warm air if their absolute humidity levels were equal.Relative humidity is cited in weather forecasts as it affects how we “feel” temperature.

[top](#20)

# Reference 
[Feature Engineering Idea Collection](https://www.kaggle.com/c/tabular-playground-series-jul-2021/discussion/250931)
[one of the main note book for which above collection borrowed the idea](https://www.kaggle.com/alexryzhkov/tps-lightautoml-baseline-with-pseudolabels)